In [ ]:
# connect drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
resume_text_path = '/content/drive/MyDrive/NLP/lemmatized_resume_text_token_ids.pkl'
job_description_path = '/content/drive/MyDrive/NLP/lemmatized_job_description_token_ids.pkl'
vocab_path = '/content/drive/MyDrive/NLP/vocab.pkl'

In [ ]:
import pickle

def load_variable(pkl_variable):
  # Open the file in binary mode
  with open(pkl_variable, 'rb') as file:
    myvar = pickle.load(file)
  return myvar

In [ ]:
pip install tensorflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 644.9/644.9 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.5/24.5 MB 44.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 60.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 57.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 60.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 3.3 MB/s eta 0:00:00


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models

In [ ]:
lemmatized_resume_text_token_ids = load_variable(resume_text_path)
lemmatized_job_description_token_ids = load_variable(job_description_path)
vocab = load_variable(vocab_path)

In [ ]:
print(len(lemmatized_resume_text_token_ids))
print(len(lemmatized_job_description_token_ids))
print(len(vocab))

10000
10000
1315


In [ ]:
# Creating the transformer encoder
import tensorflow as tf
from tensorflow.keras import layers, models

In [ ]:
# class PositionalEncoding(layers.Layer):
#     def __init__(self, max_len, d_model):
#         super().__init__()
#         pos = tf.range(max_len)[:, tf.newaxis]
#         i = tf.range(d_model)[tf.newaxis, :]
#         # Cast 'i' to tf.float32 to match the dtype of tf.pow result
#         angle_rates = 1 / tf.pow(10000., (2 * (tf.cast(i, tf.float32) // 2)) / tf.cast(d_model, tf.float32))
#         angle_rads = tf.cast(pos, tf.float32) * angle_rates

#         sines = tf.math.sin(angle_rads[:, 0::2])
#         cosines = tf.math.cos(angle_rads[:, 1::2])
#         self.pos_encoding = tf.concat([sines, cosines], axis=-1)
#         self.pos_encoding = self.pos_encoding[tf.newaxis, ...]

#     def call(self, x):
#         return x + self.pos_encoding[:, :tf.shape(x)[1], :]

class PositionalEncoding(layers.Layer):
    def __init__(self, max_len, d_model):
        super(PositionalEncoding, self).__init__()
        pos = tf.range(max_len, dtype=tf.float32)[:, tf.newaxis]
        i = tf.range(d_model, dtype=tf.float32)[tf.newaxis, :]

        angle_rates = 1 / tf.pow(10000.0, (2 * (i // 2)) / tf.cast(d_model, tf.float32))
        angle_rads = pos * angle_rates

        # Apply sin to even indices in the array; cos to odd indices
        pos_encoding = tf.where(i % 2 == 0, tf.sin(angle_rads), tf.cos(angle_rads))
        self.pos_encoding = pos_encoding[tf.newaxis, ...]

    def call(self, x):
        seq_len = tf.shape(x)[1]
        return x + self.pos_encoding[:, :seq_len, :]

In [ ]:
# def TransformerEncoderClassifier(vocab_size, d_model, num_heads, d_ff, num_layers, num_classes, max_len):
#     inputs = layers.Input(shape=(None,))
#     x = layers.Embedding(vocab_size, d_model)(inputs)
#     x = PositionalEncoding(max_len, d_model)(x)

#     for _ in range(num_layers):
#         x = layers.MultiHeadAttention(num_heads, d_model // num_heads)(x, x)
#         x = layers.LayerNormalization()(x)
#         x_ff = layers.Dense(d_ff, activation='relu')(x)
#         x = layers.Dense(d_model)(x_ff) + x
#         x = layers.LayerNormalization()(x)

#     encoder_output = x
#     x = layers.GlobalAveragePooling1D()(x)
#     logits = layers.Dense(num_classes)(x)

#     return models.Model(inputs=inputs, outputs=[logits, encoder_output])

def TransformerEncoderClassifier(vocab_size, d_model, num_heads, d_ff, num_layers, num_classes, max_len):
    inputs = layers.Input(shape=(None,), name="input_tokens")
    x = layers.Embedding(vocab_size, d_model, name="token_embedding")(inputs)
    x = PositionalEncoding(max_len, d_model)(x)

    for i in range(num_layers):
        attn_output = layers.MultiHeadAttention(num_heads=num_heads, key_dim=d_model // num_heads, name=f"mha_{i}")(x, x)
        x = layers.Add()([x, attn_output])
        x = layers.LayerNormalization(name=f"attn_norm_{i}")(x)

        ff_output = layers.Dense(d_ff, activation='relu', name=f"ffn_{i}_1")(x)
        ff_output = layers.Dense(d_model, name=f"ffn_{i}_2")(ff_output)
        x = layers.Add()([x, ff_output])
        x = layers.LayerNormalization(name=f"ffn_norm_{i}")(x)

    encoder_output = x
    pooled_output = layers.GlobalAveragePooling1D(name="global_avg_pool")(encoder_output)
    logits = layers.Dense(num_classes, name="classifier_logits")(pooled_output)

    return models.Model(inputs=inputs, outputs=[logits, encoder_output], name="TransformerEncoderClassifier")


In [ ]:
def get_embeddings(vocab, token_id):
    model = TransformerEncoderClassifier(vocab_size=2*len(vocab), d_model=256, num_heads=4, d_ff=128, num_layers=2, num_classes=2, max_len=5*token_id.shape[1])
    embedding_model = tf.keras.Model(
        inputs=model.input,
        outputs=model.output[1]  # Only encoder output
    )
    embedding = embedding_model.predict(token_id)
    return embedding

In [ ]:
def get_embeddings_1(vocab, token_id):
    model = TransformerEncoderClassifier(
        vocab_size=2000,
        d_model=256,
        num_heads=4,
        d_ff=128,
        num_layers=4,
        num_classes=2,
        max_len=5 * token_id.shape[1]
    )
    embedding_model = tf.keras.Model(
        inputs=model.input,
        outputs=model.get_layer("global_avg_pool").output  # or encoder_output if you want full sequence
    )
    embedding = embedding_model.predict(token_id)
    return embedding

In [ ]:
check_1 = get_embeddings(vocab, lemmatized_resume_text_token_ids[0])
check_2 = get_embeddings_1(vocab, lemmatized_resume_text_token_ids[0])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 342ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 455ms/step


In [ ]:
# check_1[0], check_2

In [ ]:
def embeddings(token_list_1, token_list_2):
    coupled_embeddings = []
    for i in range(len(token_list_1)):
        # embedding_1 = get_embeddings(vocab, token_list_1[i])
        # embedding_2 = get_embeddings(vocab, token_list_2[i])
        embedding_1 = get_embeddings_1(vocab, token_list_1[i])
        embedding_2 = get_embeddings_1(vocab, token_list_2[i])
        coupled_embeddings.append((embedding_1, embedding_2))
    return coupled_embeddings

In [ ]:
check = lemmatized_resume_text_token_ids[0:10] # It takes from 0-110

In [ ]:
len(check)

10

In [ ]:
# generate embeddings from [0:2500] // In this 0 in inclusive but 2500 is exclusive so it will consider [0-2499] -> total being 2500
coupled_embeddings = embeddings(lemmatized_resume_text_token_ids[0:2500], lemmatized_job_description_token_ids[0:2500])

Streaming output truncated to the last 5000 lines.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 240ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 236ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 242ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 241ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 232ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 233ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 243ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 250ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 234ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 238ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 241ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 246ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 241ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 233ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 236ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 246ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 235ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 233ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 263ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 276ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 284ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 257ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 246ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 249ms/step
1/1 ━━━━━━━━━

In [ ]:
print(len(coupled_embeddings))

2500


In [ ]:
# Saving the variable
import pickle

def save_variable(to_be_saved, file_name):
    with open(file_name, 'wb') as file:
        pickle.dump(to_be_saved, file)

In [ ]:
def generate_embeddings_and_save(start_index, end_index):
    coupled_embeddings = embeddings(lemmatized_resume_text_token_ids[start_index:end_index], lemmatized_job_description_token_ids[start_index:end_index])
    save_variable(coupled_embeddings, f'/content/drive/MyDrive/NLP/NEW_coupled_embeddings_from_{start_index}_{end_index-1}.pkl')

In [ ]:
for i in range(0,5):
    generate_embeddings_and_save(i*500, (i+1)*500)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 468ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 509ms/step


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 458ms/step


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 461ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 469ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 460ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 463ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 439ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 443ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 433ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 441ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 435ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 432ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 428ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 424ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 906ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 461ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 446ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 447ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 449ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 430ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 450ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 442ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 454ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 460ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 450ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 419ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 452ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 

In [ ]:
# save_variable(coupled_embeddings, '/content/drive/MyDrive/NLP/coupled_embeddings_from_0_2499.pkl')

In [ ]:
# save_variable(coupled_embeddings, '/content/drive/MyDrive/NLP/coupled_embeddings_from_0_2499.pkl')